In [1]:
!export TF_CPP_MIN_LOG_LEVEL=2 TF_CPP_MIN_LOG_LEVEL=2

In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import TimeDistributed, Dense
import argparse
import copy
import json
import os.path
from enum import Enum
from typing import Dict, Tuple, Union, Optional, Any
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from PIL import Image
from typing import List, Iterable, Optional, Union
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os
import pandas as pd
# physical_devices = tf.config.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], True)
from numpy import ndarray
from tensorflow import keras, Tensor
from tensorflow.keras.layers import Conv2D, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.python.keras.models import Functional
from keras_models import generate_ncp_model

IMAGE_SHAPE = (144, 256, 3)
IMAGE_SHAPE_CV = (IMAGE_SHAPE[1], IMAGE_SHAPE[0])

DEFAULT_NCP_SEED = 22222

batch_size = None
seq_len = 64
augmentation_params = None
single_step = True
no_norm_layer = False
mymodel = generate_ncp_model(seq_len, IMAGE_SHAPE, augmentation_params, batch_size, DEFAULT_NCP_SEED, single_step, no_norm_layer)

# pretrained model weights
# mymodel.load_weights('model-ncp-val.hdf5')

# custom model weights
root = "./retrain_150traj_wscheduler0.85_seed22222_lr0.001_trainloss0.00035_valloss0.00019_coreset900.h5"
mymodel.load_weights(root)

conv_layers = [layer for layer in mymodel.layers if isinstance(layer, Conv2D)]

# Get the last Conv2D layer
last_conv = conv_layers[-1]

# Get the next two layers after the last Conv2D
layer_dict = {layer.name: layer for layer in mymodel.layers}
next_layer_1 = None
next_layer_2 = None

found = False
for i, layer in enumerate(mymodel.layers):
    if layer.name == last_conv.name:
        next_layer_1 = mymodel.layers[i + 1]
        next_layer_2 = mymodel.layers[i + 2]
        break

# Sanity check
assert isinstance(next_layer_1, Flatten), f"Expected Flatten, got {type(next_layer_1)}"
assert isinstance(next_layer_2, Dense), f"Expected Dense, got {type(next_layer_2)}"

# Create new visualization model
vis_model = Model(
    inputs=mymodel.inputs[0],
    outputs=[last_conv.output, next_layer_1.output, next_layer_2.output]
)

print(vis_model.summary())


2025-05-05 19:42:11.912596: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2025-05-05 19:42:13.597199: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2025-05-05 19:42:13.597971: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2025-05-05 19:42:13.710066: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-05-05 19:42:13.712609: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA A30 computeCapability: 8.0
coreClock: 1.44GHz coreCount: 56 deviceMemorySize: 23.49GiB deviceMemoryBandwidth: 869.04GiB/s
2025-05-05 19:42:13.712625: I tensorflow/stream_executor/platform/default/dso_loa

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 144, 256, 3)]     0         
_________________________________________________________________
rescaling (Rescaling)        (None, 144, 256, 3)       0         
_________________________________________________________________
normalization (Normalization (None, 144, 256, 3)       7         
_________________________________________________________________
conv2d (Conv2D)              (None, 70, 126, 24)       1824      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 33, 61, 36)        21636     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 29, 48)        43248     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 27, 64)        2771

In [5]:
def generate_hidden_list(model: Functional, return_numpy: bool = True):
    """
    Generates a list of tensors that are used as the hidden state for the argument model when it is used in single-step
    mode. The batch dimension (0th dimension) is assumed to be 1 and any other dimensions (seq len dimensions) are
    assumed to be 0

    :param return_numpy: Whether to return output as numpy array. If false, returns as keras tensor
    :param model: Single step functional model to infer hidden states for
    :return: list of hidden states with 0 as value
    """
    constructor = np.zeros if return_numpy else tf.zeros
    hiddens = []
    print("Length of model input shape: ", len(model.input_shape))
    if len(model.input_shape)==1:
        lool = model.input_shape[0][1:]
    else:
        print("model input shape: ", model.input_shape)
        lool = model.input_shape[1:]
    print("lool: ", lool)
    for input_shape in lool:  # ignore 1st output, as is this control output
        hidden = []
        for i, shape in enumerate(input_shape):
            if shape is None:
                if i == 0:  # batch dim
                    hidden.append(1)
                    continue
                elif i == 1:  # seq len dim
                    hidden.append(0)
                    continue
                else:
                    print("Unable to infer hidden state shape. Leaving as none")
            hidden.append(shape)
        hiddens.append(constructor(hidden))
    return hiddens

In [4]:
def load_image(image_path):
    img = Image.open(image_path)
    img = img.resize(IMAGE_SHAPE_CV)
    img_array = np.array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = tf.convert_to_tensor(img_array)
    return img_array

pth = "dataset/1/Image1.png"
img = load_img(pth)
output = vis_model.predict([img])
latent_vector = output[2]
print(latent_vector)

ValueError: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {"<class 'PIL.PngImagePlugin.PngImageFile'>"}), <class 'NoneType'>

In [15]:
base_dir = "dataset"
IMAGE_SHAPE = (144, 256, 3)
IMAGE_SHAPE_CV = (IMAGE_SHAPE[1], IMAGE_SHAPE[0])

def load_image(image_path):
    img = Image.open(image_path)
    img = img.resize(IMAGE_SHAPE_CV)
    img_array = np.array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = tf.convert_to_tensor(img_array)
    return img_array


latent_space_data = []

for dir_index in range(len(os.listdir(base_dir))):
    print("Processing directory:", dir_index + 1)
    folder_path = os.path.join(base_dir, str(dir_index + 1))

    # hiddens = generate_hidden_list(model=model, return_numpy=True)
    files_length = len([file for file in os.listdir(folder_path) if '.png' in file])

    for frame_index in range(files_length - 1):
        image_path = f"{folder_path}/Image{frame_index + 1}.png"
        img = load_image(image_path)
        print(image_path)

        output = vis_model.predict([img])
        latent_vector = output[2]
        # hiddens = output[1:]
        # preds = output[0][0]
        # vx, vy, vz, omega_z = preds[0], preds[1], preds[2], preds[3]

        # prediction_data.append([vx, vy, vz, omega_z])
        break
    break

df = pd.DataFrame(prediction_data, columns=['vx', 'vy', 'vz', 'omega_z'])

# Save to CSV
# df.to_csv("prediction_output.csv", index=False)


Processing directory: 1
dataset/1/Image1.png
(1, 6, 13, 16)
